<a href="https://colab.research.google.com/github/Nubiancodingdelight/ACS-Project-Repository-/blob/main/Generate_Matrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebooks generates matrices as numpy arrays.
Created By: Lorrayya Williams
Updated On: 4/13/2025

In [1]:
#mounts google drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
#set path
%cd /content/drive/MyDrive/ACS_Research/VISDB_Data/

/content/drive/MyDrive/ACS_Research/VISDB_Data


INSTALL

In [3]:
!pip install pysam

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.0/26.0 MB 53.0 MB/s eta 0:00:00


Imports

In [4]:
#imports
import numpy as np
import subprocess
import shlex
import pysam
import re
import tensorflow as tf
import re
import math
import statistics as stat


# Functions




In [ ]:

#def align_seq(seq, start, stop,dir):


In [20]:
import pysam
from pysam import AlignmentHeader, AlignedSegment
def create_bam_from_sequence(output_bam_path, dna_sequence, reference_name, start_pos):
  """
  Creates a BAM file from a DNA sequence and its coordinates.

  Args:
      output_bam_path (str): Path to save the BAM file (e.g., "output.bam").
      dna_sequence (str): DNA sequence (e.g., "ATCGATCG").
      reference_name (str): Reference sequence name (e.g., "chr1").
      start_pos (int): 1-based start position on the reference.
  """
  # 1. Create a BAM header
  header = AlignmentHeader.from_references(
      [reference_name],  # List of reference names
      [len(dna_sequence)]  # Lengths of references
  )

  # 2. Open a BAM file for writing
  with pysam.AlignmentFile(output_bam_path, "wb", header=header) as bam_file:
    # 3. Create an aligned segment (read)
    read = AlignedSegment(header)
    read.query_name = "read1"  # Read ID
    read.query_sequence = dna_sequence  # DNA sequence
    read.flag = 0  # No flags set (0 means mapped)
    read.reference_id = 0  # Index of reference in header (0 = first one)
    read.reference_start = start_pos - 1  # 0-based position
    read.mapping_quality = 60  # High mapping quality
    read.cigar = [(pysam.CMATCH, len(dna_sequence))]  # CIGAR string (exact match)
    #print(read.cigar)
    #print(read)
    # 4. Write the read to the BAM file
    bam_file.write(read)


Generate BAM files

HBV

In [7]:
#load hbv data
import pandas as pd
hbv_data = pd.read_csv('Spliced_Data_HBV.csv')
hbv_data.head()

,Unnamed: 0,virus,virus_ref,begin_ref,stop_ref,human_ref,begin_breakpoint,stop_breakpoint,spliced_seq,viral_seq,human_seq_upstream,human_seq_downstream
0,0,HBV,X70185.1,420,437,GRCh37/hg19,1720,1603,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...,NaN,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...,NNNNNNNNNNNNNNNN
1,1,HBV,X70185.1,97732036,97732056,GRCh37/hg19,1246,1390,TATCAATTAAACAAATCATTGCTGTGTCCTGCAGTTCAGGGTTCTG...,TTTGTGGCTCCTCTGCCGATCCATACTGCGGAACTCCTAGCCGCTT...,TATCAATTAAACAAATCATTGCTGTGTCCTGCAGTTCAGGGTTCTG...,gtaatttgtaatgaacagatattcatttCtttttttattattatta...
2,2,HBV,X70185.1,194161891,194161910,GRCh37/hg19,1603,1664,CTTTAATTAATTGTTAGTATGACTAAAAATTTATTTCACAAAGTTG...,GTTGCATGGAGACCACCGTGAACGCCCATCAGATCCTGCCCAAGGT...,CTTTAATTAATTGTTAGTATGACTAAAAATTTATTTCACAAAGTTG...,AGTGTTTGCTTTTTAAAATGTTAAGATTAAATCATATTGTTACttt...
3,3,HBV,X70185.1,171450967,171450816,GRCh37/hg19,1623,1717,ctagtaagccagtgtttcctatgttggAACGCCCATCAGATCCTGC...,AACGCCCATCAGATCCTGCCCAAGGTCTTACATAAGAGGACTCTTG...,ctagtaagccagtgtttcctatgttgg,caactatctccctgatcactgtaattggtccagagtaggacaaatg...
4,4,HBV,X70185.1,171450967,171450816,GRCh37/hg19,1638,1717,ctagtaagccagtgtttcctatgttgggtaaacacatttgcagtgg...,CTGCCCAAGGTCTTACATAAGAGGACTCTTGGACTCTCAGCAATGT...,ctagtaagccagtgtttcctatgttgggtaaacacatttgcagtgg...,AGACTTTATATATGGCTACCTCTTGACCTCCAAATCCCATCCTCCT...


In [18]:
# prompt: remove first row from hbv_Data

hbv_data = hbv_data.iloc[1:]


In [24]:
for index, row in hbv_data.iterrows():
  create_bam_from_sequence('/content/drive/MyDrive/ACS_Research/VISDB_Data/Aligned_Sequences/'  +str(row['virus'])+str(index)+'.bam',row['spliced_seq'], 'hg19', row['begin_breakpoint'])

[(0, 345)]
read1	0	hg19	1246	60	345M	*	0	0	TATCAATTAAACAAATCATTGCTGTGTCCTGCAGTTCAGGGTTCTGCATCTTTGGtgtcttaatctgctgtatgctgctatTTTGTGGCTCCTCTGCCGATCCATACTGCGGAACTCCTAGCCGCTTGTTTTGCTCGCAGCCGGTCTGGAGCAAAGCTCATCGGAACTGACAATTCTGTCGTCCTCTCGCGGAAATATACATCGTTTCCATGGCTGCTAGGCTGTAgtaatttgtaatgaacagatattcatttCtttttttattattattatacgttaagttctagggtacatgtgcacaacgggcaggtttgttacataggtatacatgtgccatgttggttt	None	[]
[(0, 262)]
read1	0	hg19	1603	60	262M	*	0	0	CTTTAATTAATTGTTAGTATGACTAAAAATTTATTTCACAAAGTTGCTGAAAACAAACATTTGGTTGCATGGAGACCACCGTGAACGCCCATCAGATCCTGCCCAAGGTCTTACATAAGAGGACTAGTGTTTGCTTTTTAAAATGTTAAGATTAAATCATATTGTTACtttatttttttctaattaatattttatatGAAGTGTTCTTCTATTAAATGGGTTGATGTCACCATAAAATATGAATAGAATAGCTATCTTGTCA	None	[]
[(0, 295)]
read1	0	hg19	1623	60	295M	*	0	0	ctagtaagccagtgtttcctatgttggAACGCCCATCAGATCCTGCCCAAGGTCTTACATAAGAGGACTCTTGGACTCTCAGCAATGTCAACGACCGACCTTGAGGCCTACTTCAAAGACTGcaactatctccctgatcactgtaattggtccagagtaggacaaatgcctcaagttaggccTCTCTGAGACTTTATATATGGCTACCTCTTGACCTCCAAATCCCATCCTCCTGCCCTGTCATTT

In [11]:
%pwd

'/content/drive/MyDrive/ACS_Research/VISDB_Data/Aligned_Sequences'

Generate SAM file

#Matrix Generator 1 -- DeepHBV


In [ ]:
import numpy as np
import os
from scipy.io import loadmat
# from array import array
# from util import seq_matrix

def seq_matrix(seq_list, dim):  # One Hot Encoding
    tensor = np.zeros((len(seq_list), dim, 4))
    for i in range(len(seq_list)):
        seq = seq_list[i]
        j = 0
        for s in seq:
            # if s == 'A' or s == 'a':
            #    tensor[i][j] = [1, 0, 0, 1, 0, 1, 0, 0]
            # if s == 'T' or s == 't':
            #    tensor[i][j] = [0, 1, 0, 0, 1, 0, 0, 0]
            # if s == 'C' or s == 'c':
            #    tensor[i][j] = [0, 0, 1, 0, 0, 1, 0, 1]
            # if s == 'G' or s == 'g':
            #    tensor[i][j] = [0, 0, 0, 1, 0, 0, 1, 1]
            # if s == 'N':
            #    tensor[i][j] = [0, 0, 0, 0, 0, 0, 0, 0]
            if s == 'A' or s == 'a':
                tensor[i][j] = [1, 0, 0, 0]
            if s == 'T' or s == 't':
                tensor[i][j] = [0, 1, 0, 0]
            if s == 'C' or s == 'c':
                tensor[i][j] = [0, 0, 1, 0]
            if s == 'G' or s == 'g':
                tensor[i][j] = [0, 0, 0, 1]
            if s == 'N':
                tensor[i][j] = [0, 0, 0, 0]
            j += 1
    return tensor
def fasta_to_matrix():
    #seq_name_file = ['DNA_neg_Test.mat',
    #                 'DNA_pos_Test.mat']  # data file
    #mat file is a matlab file

    #print(seq_name_file)
    dim = 75

#POSITIVE SEQUENCE TRAIN
    seq = POS_SEQ
    print('pos__Train_starting!')
    global Data_pos_Train
    Data_pos_Train = seq_matrix(seq, dim)
    # global Data_pos_Train
    print('pos__Train_ending!')

#NEGATIVE SEQUENCE
    seq = NEG_SEQ
    print('neg_Train_starting!')
    global Data_neg_Train
    Data_neg_Train = seq_matrix(seq, dim)
    # global Data_neg_Train
    print('neg_Train_ending!')
    Data_Train = np.concatenate([Data_pos_Train, Data_neg_Train])  # Train data
    Label_Train = np.concatenate([np.ones(len(Data_pos_Train)), np.zeros(len(Data_neg_Train))])  # Trian_label
    #Data_Test = np.concatenate([Data_pos_Test, Data_neg_Test])  # Test data
    #print(Data_Test)
    #Label_Test = np.concatenate([np.ones(len(Data_pos_Test)), np.zeros(len(Data_neg_Test))])  # Test label
    #print(Label_Test)
    np.save('Data/Simulation_Data1_Train', Data_Train)
    np.save('Data/Simulation_Data1_Label_Train', Label_Train)
    #np.save('Data_Test', Data_Test)
    #np.save('Label_Test', Label_Test)

if __name__ == '__main__':
    fasta_to_matrix( )

# Matrix Generator 2 -- Novel Matrix

In [ ]:
#matrix row generator from CIGAR, SEQuence, Position of Insert
def matrix_row(CIG, SE, PO, INS):
  matrix_list= []
  label_list =[]
  bases = ['A','T','C','G']
  seq_list= list(SEQ)
  cigar_list = re.split(r'(\d+)', CIGAR)[1::]
  tensor= np.zeros((1,75, 4))
  if int(POS) in range(24020700,24020710):
    label_list.append(1)
  else:
    label_list.append(0)

  #print(len(seq_list))
  #FIX LOOP
  if len(cigar_list) ==2:
    #if it is a matched a sequence of 1's based on the base are added
    if cigar_list[1] == 'M':
      i= 0
      for j in range(int(cigar_list[0])):
        num_list = [0,0,0,0]
        try:
          num_list[bases.index(seq_list[j])] = 1
          #print(num_list)
          tensor[i][j]= num_list
          #print(tensor[i][j])
        except ValueError:
          tensor[i][j]= num_list


    else:
      for j in range(len(seq_list)):
        num_list = [0,0,0,0]

#  return tensor, label_list

  else:
    length = 0
    i = 0
    for time in range(int(len(cigar_list)/2)):
      if cigar_list[(2*time)+1] == 'M':
        for j in range(length,length + int(cigar_list[(2*time)])):
          num_list = [0,0,0,0]
          try:
            num_list[bases.index(seq_list[j])] = 1
            tensor[i][j]= num_list
          except ValueError:
            tensor[i][j]= num_list
        length += int(cigar_list[(2*time)])
  return tensor, label_list


In [ ]:
def matrix_generator_dos (samfile, INS, Matrix_Name ='Matrix_data', Label_Name='Label_data', save_file_path=os.getcwd()):
  final_matrix = []
  label_list = []
  COUNT =0
  #loops through each line of the sam file and extracts the CIGAR Value, Sequence, and Current Position
  for line in samfile:
    line_list = str(line).split("\t")
    CIGAR = line_list[5]
    SEQ = line_list[9]
    POS = line_list[3]
    final_matrix_temp, label_list_temp = matrix_row(CIGAR, SEQ, POS, INS)
    if len(final_matrix) == 0:
      final_matrix = final_matrix_temp
    else:
      final_matrix = np.concatenate([final_matrix, final_matrix_temp])

  label_list += label_list_temp

  #saves matrix to specified path
  os.chdir(save_file_path)
  Data = np.asarray(final_matrix)
  Label = np.asarray(label_list)
  np.save(Matrix_Name, Data[:][:][:])                                                                                                                                                                                                                     ', Data[:4405][:][:])
  np.save(Label_name, Label[:])

# Matrix Generator 3 -- 3D Matrix

> Add blockquote



In [ ]:
def matrix_generator_tres(samfile, int_start, int_end, Matrix_Name ='Matrix_data', Label_Name='Label_data', save_file_path=os.getcwd()):
  for line in samfile:
    line_list = str(line).split("\t")
    CIGAR =  re.split(r'(\d+)', line_list[5])[1::]
    SEQ = list(line_list[9])
    POS = line_list[3]
    DIR = 1 if line_list[8][0] == '-' else 0

    if len(CIGAR) ==2:
    #if it is a matched a sequence of 1's based on the base are added
      if CIGAR[1] == 'M':
        mat = 1
        for i in range(len(SEQ)):
          mapping_list.append([SEQ[i], mat, int(POS) + i ,DIR, COUNT])

      else:
        mat=0
        for i in range(len(SEQ)):
          mapping_list.append([SEQ[i], mat, int(POS) + i ,DIR, COUNT])
    else:
      length = 0
      i = 0
      for time in range(int(len(CIGAR)/2)):
        if CIGAR[(2*time)+1] == 'M':
          for j in range(length,length + int(CIGAR[(2*time)])):
            mapping_list.append([SEQ[i], mat, int(POS) + i ,DIR, COUNT])
        else:
          for j in range(length,length + int(CIGAR[(2*time)])):
            mapping_list.append([SEQ[i], mat, int(POS) + i ,DIR, COUNT])
        COUNT += 1

  #order a list lists
  df = pd.DataFrame(mapping_list, columns = ['Base', 'Match', 'Position', 'Direction', 'Row_count'])
  df = df.sort_values(by=['Position'])
  #df = df.reset_index(drop=True)
  df = df.tail(-1).drop_duplicates()
  df = df.reset_index(drop=True)
  three_five = df[df['Direction'] ==0]
  five_three = df[df['Direction'] ==1]

    #create tensor
  tensor = np.zeros((len(df),150,  4))
  label = np.zeros(len(df))
  label_list= [0] * len(df)

  #
  col_count = 0
  curr_pos = 0
  end_pos = 0




for row in range(1,len(df)):
  bases = ['A','T','C','G']
  match_dict ={'A':'T', 'T':'A', 'C':'G', 'G':'C'}
  #print(row)
  #print(df['Position'][row])
  curr_pos = df['Position'][row]

#3' -->5' Direction
  if df['Direction'][row] == 0:
    #checks to see if it matches with alignment genome
    if three_five['Base'][row] in bases and three_five['Match'][row] == 1:
      tensor[row][col_count][bases.index(three_five['Base'][row])] = 1

      #checks if the bases match with each other
      #print(five_three[five_three['Position']== three_five['Position'][row]].Base.values[0])
      if five_three[five_three['Position']== three_five['Position'][row]].Base.values[0] == match_dict[three_five['Base'][row]]:
        tensor[row][col_count][bases.index(three_five['Base'][row])] =3

    #checks if it matches with each other but not with alignment genome
    elif five_three['Position'][row]['Base'] == match_dict[three_five['Base'][row]]:
        tensor[row][col_count][bases.index(three_five['Base'][row])]=2

#5'-->3' Direction
  else:
    #checks to see if it matches with alignment genome
    if five_three['Base'][row] in bases and five_three['Match'][row] == 1:
      tensor[row][col_count][bases.index(five_three['Base'][row])] = 1

      #checks if the bases match with each other
      #print( match_dict[five_three['Base'][row]])

      #print(three_five[three_five['Position']== five_three['Position'][row]].Base.item())
      if three_five[three_five['Position']== five_three['Position'][row]].Base.values[0] == match_dict[five_three['Base'][row]]:
        tensor[row][col_count][bases.index(five_three['Base'][row])] =3

    #checks if it matches with each other but not with alignment genome
    elif three_five['Position'][row]['Base'] == match_dict[five_three['Base'][row]]:
        tensor[row][col_count][bases.index(five_three['Base'][row])]=2

  if curr_pos == end_pos:
    if int(curr_pos) in range(int_start,int_end):
      label_list.append(1)
    else:
      label_list.append(0)
    end_pos +=75
    col_count +=1
    if col_count == 150:
      col_count =0

  elif end_pos == 0:
    end_pos= curr_pos +75
    if int(curr_pos) in range(int_start,int_end):
      label_list.append(1)
    else:
      label_list.append(0)



In [ ]:
for line in samfile:
  line_list = str(line).split("\t")
  CIGAR =  re.split(r'(\d+)', line_list[5])[1::]
  SEQ = list(line_list[9])
  POS = line_list[3]
  DIR = 1 if line_list[8][0] == '-' else 0


  if len(CIGAR) ==2:
  #if it is a matched a sequence of 1's based on the base are added
    if CIGAR[1] == 'M':
      mat = 1
      for i in range(len(SEQ)):
        mapping_list.append([SEQ[i], mat, int(POS) + i ,DIR, COUNT])

    else:
      mat=0
      for i in range(len(SEQ)):
        mapping_list.append([SEQ[i], mat, int(POS) + i ,DIR, COUNT])
  else:
    length = 0
    i = 0
    for time in range(int(len(CIGAR)/2)):
      if CIGAR[(2*time)+1] == 'M':
        for j in range(length,length + int(CIGAR[(2*time)])):
          mapping_list.append([SEQ[i], mat, int(POS) + i ,DIR, COUNT])
      else:
        for j in range(length,length + int(CIGAR[(2*time)])):
          mapping_list.append([SEQ[i], mat, int(POS) + i ,DIR, COUNT])
      COUNT += 1

In [ ]:
#order a list lists
df = pd.DataFrame(mapping_list, columns = ['Base', 'Match', 'Position', 'Direction', 'Row_count'])
df = df.sort_values(by=['Position'])
#df = df.reset_index(drop=True)
df = df.tail(-1).drop_duplicates()
df = df.reset_index(drop=True)
three_five = df[df['Direction'] ==0]
five_three = df[df['Direction'] ==1]

In [ ]:
#create tensor
tensor = np.zeros((len(df),150,  4))
label = np.zeros(len(df))
label_list= [0] * len(df)

#
col_count = 0
curr_pos = 0
end_pos = 0

In [ ]:
#generate Matrix

for row in range(1,len(df)):
  bases = ['A','T','C','G']
  match_dict ={'A':'T', 'T':'A', 'C':'G', 'G':'C'}
  #print(row)
  #print(df['Position'][row])
  curr_pos = df['Position'][row]

#3' -->5' Direction
  if df['Direction'][row] == 0:
    #checks to see if it matches with alignment genome
    if three_five['Base'][row] in bases and three_five['Match'][row] == 1:
      tensor[row][col_count][bases.index(three_five['Base'][row])] = 1

      #checks if the bases match with each other
      #print(five_three[five_three['Position']== three_five['Position'][row]].Base.values[0])
      if five_three[five_three['Position']== three_five['Position'][row]].Base.values[0] == match_dict[three_five['Base'][row]]:
        tensor[row][col_count][bases.index(three_five['Base'][row])] =3

    #checks if it matches with each other but not with alignment genome
    elif five_three['Position'][row]['Base'] == match_dict[three_five['Base'][row]]:
        tensor[row][col_count][bases.index(three_five['Base'][row])]=2

#5'-->3' Direction
  else:
    #checks to see if it matches with alignment genome
    if five_three['Base'][row] in bases and five_three['Match'][row] == 1:
      tensor[row][col_count][bases.index(five_three['Base'][row])] = 1

      #checks if the bases match with each other
      #print( match_dict[five_three['Base'][row]])

      #print(three_five[three_five['Position']== five_three['Position'][row]].Base.item())
      if three_five[three_five['Position']== five_three['Position'][row]].Base.values[0] == match_dict[five_three['Base'][row]]:
        tensor[row][col_count][bases.index(five_three['Base'][row])] =3

    #checks if it matches with each other but not with alignment genome
    elif three_five['Position'][row]['Base'] == match_dict[five_three['Base'][row]]:
        tensor[row][col_count][bases.index(five_three['Base'][row])]=2

  if curr_pos == end_pos:
    if int(curr_pos) in range(24020700,24020710):
      label_list.append(1)
    else:
      label_list.append(0)
    print("Here")
    end_pos +=75
    col_count +=1
    if col_count == 150:
      col_count =0

  elif end_pos == 0:
    print("Here")
    end_pos= curr_pos +75
    if int(curr_pos) in range(24020700,24020710):
      label_list.append(1)
    else:
      label_list.append(0)


In [ ]:
#save tensor

Data = np.asarray(tensor)
Label = np.asarray(label_list)
np.save('Data_Training_Exp3', Data[:22568][:][:])
np.save('Data_Test_Exp3', Data[22568:][:][:])
np.save('Label_Training_Exp3', Label[:22568])
np.save('Label_Test_Exp3', Label[22568:])
